<a href="https://colab.research.google.com/github/SG-Hangaan/Kaggle-Competition-Solutions/blob/main/Kaggle_Spaceship_Titanic_Finals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import tensorflow and checking the versions
import tensorflow as tf
print(tf.__version__)

2.14.0


## **Download Kaggle Challenge Dataset**

In [ ]:
#Install dependencies
!pip install kaggle

In [ ]:
!pwd

/content


In [ ]:
#Getting Kaggle.JSON file from Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


:Extracting the Dataset

In [ ]:
!unzip 'spaceship-titanic.zip'

Archive:  spaceship-titanic.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


Removing ZIP File

In [ ]:
!rm -rf 'spaceship-titanic.zip'

# **Develop a model**
Importing important packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

# Input data files are available in the "../input/" directory.
from subprocess import check_output
print(check_output(["ls", "../content"]).decode("utf8"))

content
drive
sample_data
sample_submission.csv
test.csv
train.csv



** Begin by reading and Reshape the dataset**

In [ ]:
# Read training and test data files
training_data = pd.read_csv("train.csv")
training_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


**Exploratory Data Analysis**

In [ ]:
# printing column names
print(f"Column bames in dataset are: {list(training_data.columns)} \n")

# printing shape of data
print(f"Shape of dataset is : {training_data.shape} \n")

# printing data tyoe of every column in dataset
print(f"Description of columns: \n{training_data.dtypes} \n")

# checking missing values in every column
print(f"Missing Values : \n{training_data.isnull().sum()} \n")

Column bames in dataset are: ['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Transported'] 

Shape of dataset is : (8693, 14) 

Description of columns: 
PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object 

Missing Values : 
PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64 



Handle missing values after train test split.

# **Data Preprocessing**

Handling Missing Values

In [ ]:
# replacing the missing values in "Age" column with mean value
training_data['Age'].fillna(training_data['Age'].mean(),inplace=True)

# replacing the missing values in "RoomService" column with mean value
training_data['RoomService'].fillna(training_data['RoomService'].mean(), inplace=True)

# replacing the missing values in "FoodCourt" column with mean value
training_data['FoodCourt'].fillna(training_data['FoodCourt'].mean(), inplace=True)

# replacing the missing values in "ShoppingMall" column with mean value
training_data['ShoppingMall'].fillna(training_data['ShoppingMall'].mean(), inplace=True)

# replacing the missing values in "Spa" column with mean value
training_data['Spa'].fillna(training_data['Spa'].mean(), inplace=True)

# replacing the missing values in "VRDeck" column with mean value
training_data['VRDeck'].fillna(training_data['VRDeck'].mean(), inplace=True)

# replacing the missing values in "HomePlanet" column with mode value
training_data['HomePlanet'].fillna(training_data['HomePlanet'].mode()[0], inplace=True)

# replacing the missing values in "CryoSleep" column with mode value
training_data['CryoSleep'].fillna(training_data['CryoSleep'].mode()[0], inplace=True)

# replacing the missing values in "Cabin" column with mode value
training_data['Cabin'].fillna(training_data['Cabin'].mode()[0], inplace=True)

# replacing the missing values in "Destination" column with mode value
training_data['Destination'].fillna(training_data['Destination'].mode()[0], inplace=True)

# replacing the missing values in "VIP" column with mode value
training_data['VIP'].fillna(training_data['VIP'].mode()[0], inplace=True)

# checking missing values in every column
print(f"Missing Values: \n{training_data.isnull().sum()} \n")


Missing Values: 
PassengerId       0
HomePlanet        0
CryoSleep         0
Cabin             0
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name            200
Transported       0
dtype: int64 



Handling Categorical Variables

In [ ]:
# printing unique values in categorical columns
print(f"Unique Values in HomePlanet: {training_data['HomePlanet'].unique()} \n")
print(f"Unique Values in CryoSleep: {training_data['CryoSleep'].unique()} \n")
print(f"Unique Values in Cabin: {training_data['Cabin'].unique()} \n")
print(f"Unique Values in Destination: {training_data['Destination'].unique()} \n")
print(f"Unique Values in VIP: {training_data['VIP'].unique()} \n")

Unique Values in HomePlanet: ['Europa' 'Earth' 'Mars'] 

Unique Values in CryoSleep: [False  True] 

Unique Values in Cabin: ['B/0/P' 'F/0/S' 'A/0/S' ... 'G/1499/S' 'G/1500/S' 'E/608/S'] 

Unique Values in Destination: ['TRAPPIST-1e' 'PSO J318.5-22' '55 Cancri e'] 

Unique Values in VIP: [False  True] 



Target variable is "Transported". Separate target from features and perform train test split.

In [ ]:
model_1_training_data = training_data.copy()

# Target
y_1 = model_1_training_data['Transported']

# Single Feature
X_1 = model_1_training_data[['Spa']]

X_train, X_test, y_train, y_test = train_test_split(X_1, y_1, random_state=42)

In [ ]:
# Replace missing values with the median
imputer = SimpleImputer(strategy='median')
# Fit imputer to the indepedent variable
# using only the training data
imputer.fit(X_train)
# Replace missing values in the training and test data
X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

Fit a basic Logistic Regression model

In [ ]:
model_1 = LogisticRegression()
model_1.fit(X_train, y_train)

LogisticRegression()

LogisticRegression()

## **Evaluate model performance**

In [ ]:
train_preds = model_1.predict(X_train)
test_preds = model_1.predict(X_test)

train_score = accuracy_score(y_train, train_preds)
test_score = accuracy_score(y_test, test_preds)

print('Train score:', train_score)
print('Test score:', test_score)

Train score: 0.6277036355269213
Test score: 0.6214351425942962


## **Create submission predictions**

Import testing data

In [ ]:


test_df = pd.read_csv('test.csv')
test_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


Repeat same preprocessing steps as before

In [ ]:
test_X = test_df[['Spa']]

In [ ]:
# Impute using fitted imputer
test_X = imputer.transform(test_X)

Create final predictions

In [ ]:
final_preds = model_1.predict(test_X)

Save predictions

In [ ]:
# Add predictions to the test dataframe
test_df['Transported'] = final_preds
# Isolate the columns we want in our submission
submission_df = test_df[['PassengerId', 'Transported']]

In [ ]:
submission_df.shape

(4277, 2)

In [ ]:
submission_df.to_csv('submission.csv', index=False)

In [ ]:
#Submitting the results on Kaggle
!kaggle competitions submit -c spaceship-titanic -f submission.csv -m "Message"

100% 55.1k/55.1k [00:01<00:00, 56.5kB/s]
Successfully submitted to Spaceship Titanic